# Pushing the custom model to huggingface hub

The following notebook is provided as a reference for how I pushed the custom model code to huggingface. Note that it should not be run, as the model and config has already been registered.

Reference: https://huggingface.co/docs/transformers/v4.39.1/custom_models#sending-the-code-to-the-hub

```
.
└── scratch_transformer_model
    ├── __init__.py
    ├── configuration_scratch_transformer.py
    ├── modeling_scratch_transformer.py
    └── model.py
```

To share your model with the community, follow those steps: first import the ScratchTransformer model and config from the newly created files:

In [1]:
! export PYTHONPATH=.

In [2]:
from huggingface_upload.scratch_transformer_model.configuration_scratch_transformer import ScratchTransformerConfig
from huggingface_upload.scratch_transformer_model.model_scratch_transformer import ScratchTransformerModel

Then you have to tell the library you want to copy the code files of those objects when using the save_pretrained method and properly register them with a given Auto class (especially for models) by running:

In [3]:
ScratchTransformerConfig.register_for_auto_class()
ScratchTransformerModel.register_for_auto_class("AutoModel")

Next, let’s create the config and models, and load the model with the pretrained weights.

In [4]:
import torch
from train import get_config
from config import get_weights_file_path
from tokenizers import Tokenizer

cfg = get_config()
cfg['model_folder'] = 'weights'
cfg['tokenizer_file'] = 'vocab/tokenizer{0}.json'
cfg['preload'] = '29'

tokenizer_src = Tokenizer.from_file(str('vocab/tokenizeren.json'))
tokenizer_tgt = Tokenizer.from_file(str('vocab/tokenizerit.json'))

scratch_transformer_config = ScratchTransformerConfig(
                                                    src_vocab_size=tokenizer_src.get_vocab_size(), 
                                                    tgt_vocab_size=tokenizer_tgt.get_vocab_size(), 
                                                    )
scratch_transformer = ScratchTransformerModel(scratch_transformer_config)

scratch_transformer.config.decoder_start_token_id = tokenizer_tgt.token_to_id("[SOS]")
scratch_transformer.config.pad_token_id = tokenizer_tgt.token_to_id("[PAD]")
scratch_transformer.config.eos_token_id = tokenizer_tgt.token_to_id("[EOS]")

# This is where I load the model from my checkpoint weights file!!
model_filename = get_weights_file_path(cfg, cfg['preload'])
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
state = torch.load(model_filename, map_location=torch.device(device))
scratch_transformer.model.load_state_dict(state['model_state_dict'])

<All keys matched successfully>

Now to send the model to the Hub, make sure you are logged in. Either run in a terminal:

```
huggingface-cli login
```

Or in the notebook (** note that there are sometimes issues with vscode with this, here's a [workaround](https://github.com/huggingface/huggingface_hub/issues/752#issuecomment-1063793855)):

In [5]:
from huggingface_hub import notebook_login

notebook_login()

You can then push to your own namespace (or an organization you are a member of) like this:

In [6]:
scratch_transformer.push_to_hub("gretakate/english-to-italian-transformer-from-scratch")

model.safetensors:   0%|          | 0.00/302M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/gretakate/english-to-italian-transformer-from-scratch/commit/e857b0518ae6d8dd6d52f7f8575d85a39ee7f81f', commit_message='Upload model', commit_description='', oid='e857b0518ae6d8dd6d52f7f8575d85a39ee7f81f', pr_url=None, pr_revision=None, pr_num=None)

On top of the modeling weights and the configuration in json format, this also copied the modeling and configuration .py files in the folder scratch_transformer_model and uploaded the result to the Hub. You can check the result in this model repo.

Using the transformer model


References:
- https://huggingface.co/docs/transformers/v4.39.1/en/model_doc/encoder-decoder#transformers.EncoderDecoderModel
- https://colab.research.google.com/drive/1WIk2bxglElfZewOHboPFNj8H44_VAyKE?usp=sharing#scrollTo=ZwQIEhKOrJpl 